In [2]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/arcane/')

import numpy as np
import pandas as pd
import json

from sql.articles.MongoDBArticle import MongoDBArticle

from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [12]:
with PostgresDatabaseOperation() as cursor:
    get_sql = """SELECT DISTINCT ca.article_id, published_at, financial_news, relevant_for_india, india_flag, source_name
                 FROM candidate_articles ca 
                 LEFT JOIN source_id_characteristics s ON ca.source_id = s.source_id
                 LEFT JOIN llm_article_attributes laa ON ca.article_id = laa.article_id
                 WHERE ca.content_type = 'ARTICLE'
                 """
    cursor.execute(get_sql)
    results = cursor.fetchall()

In [14]:
df = pd.DataFrame(results, columns = ['article_id', 'published_at', 'financial_news', 'relevant_for_india', 'india_flag', 'source_name'])

In [18]:
articles = MongoDBArticle.fetch_documents_by_ids(list(df.article_id.unique()))

In [20]:
articles = {str(k['_id']): k for k in articles}

In [25]:
df['title'] = [articles[x]['title'] for x in df['article_id']]

In [34]:
df[(df.source_name == 'yahoo-finance') & (df.title.str.contains('BGHL'))]

,article_id,published_at,financial_news,relevant_for_india,india_flag,source_name,title


In [54]:
thousand_titles = ';'.join(df.sample(1000).title.values)

2024-02-15 12:20:44,268| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [55]:
thousand_titles

'A.K.Capital Ser Standalone December 2023 Net Sales at Rs 31.44 crore, up 7.24% Y-o-Y;Inox Wind to execute India\'s single largest wind project order of 1,500 MW from CESC;L&T construction wins "large" orders in India and the Middle East, its third in three days;Italian Edibles lists at 19% discount over IPO price on NSE SME;Uttar Pradesh govt targets 1 million tonne capacity and 120,000 jobs;Indices trade flat, fiscal deficit target set at 5.1% of GDP for FY25;HDFC Bank announces raising $750 mn through multi-year overseas bonds;MC Explains | All you need to know about Chinese stock market rout;UBS CEO says more capital not the way to make it safer;ISL Consulting standalone net profit rises 3540.00% in the December 2023 quarter | Capital Market News - Business Standard;Bosch pushes back margin target, warns of further job cuts;Docmode Health Tech IPO allotment finalised; here\'s how to check allotment status | Mint;Growth pattern shows all sectors of economy are revving up, says FM Si

2024-02-15 12:20:54,330| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-15 12:21:04,408| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-15 12:21:14,481| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-15 12:21:24,549| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the

In [92]:
responses_from_gpt = '''{"title": "A.K.Capital Ser Standalone December 2023 Net Sales at Rs 31.44 crore, up 7.24% Y-o-Y", "category": "corporate_developments"},
  {"title": "Inox Wind to execute India's single largest wind project order of 1,500 MW from CESC", "category": "corporate_developments"},
  {"title": "L&T construction wins \"large\" orders in India and the Middle East, its third in three days", "category": "corporate_developments"},
  {"title": "Italian Edibles lists at 19% discount over IPO price on NSE SME", "category": "market_strategies"},
  {"title": "Uttar Pradesh govt targets 1 million tonne capacity and 120,000 jobs", "category": "policy_impact"},
  {"title": "Indices trade flat, fiscal deficit target set at 5.1% of GDP for FY25", "category": "policy_impact"},
  {"title": "HDFC Bank announces raising $750 mn through multi-year overseas bonds", "category": "corporate_developments"},
  {"title": "All you need to know about Chinese stock market rout", "category": "market_strategies"},
  {"title": "UBS CEO says more capital not the way to make it safer", "category": "corporate_developments"},
  {"title": "ISL Consulting standalone net profit rises 3540.00% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Bosch pushes back margin target, warns of further job cuts", "category": "corporate_developments"},
  {"title": "Docmode Health Tech IPO allotment finalised; here's how to check allotment status", "category": "market_strategies"},
  {"title": "Growth pattern shows all sectors of economy are revving up, says FM Sitharaman", "category": "policy_impact"},
  {"title": "ITC's largest shareholder BAT is working towards monetising some of its stake", "category": "corporate_developments"},
  {"title": "Will INOVIQ (ASX:IIQ) Spend Its Cash Wisely?", "category": "corporate_developments"},
  {"title": "Interim Budget: Disinvestment, monetisation target set at Rs 50,000 crore", "category": "policy_impact"},
  {"title": "Why Apple's iconic Super Bowl ad still resonates 4 decades later", "category": "corporate_developments"},
  {"title": "Mission Shakti: Under longest-serving woman FM, gender Budget hits a high", "category": "policy_impact"},
  {"title": "India is considering to extend parboiled rice tax in risk to world supply", "category": "policy_impact"},
  {"title": "So what do I do if I need to get VC funding? – Varsity by Zerodha", "category": "startup_ecosystem"},
  {"title": "Religare shares worth Rs 338 crore change hands; Burman family likely buyers", "category": "market_strategies"},
  {"title": "Budget 2024: What to expect from Sitharaman's interim budget presentation", "category": "policy_impact"},
  {"title": "Selan Explore Standalone December 2023 Net Sales at Rs 43.57 crore, up 50.19% Y-o-Y", "category": "corporate_developments"},
  {"title": "Philippines' San Miguel-led group makes top offer in airport bid", "category": "corporate_developments"},
  {"title": "Olectra Greentech Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "The Home Depot Named on Fortune’s 2024 Most Admired Companies List", "category": "corporate_developments"},
  {"title": "AMS Polymers standalone net profit rises 480.00% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Budget 2024: Capex outlay raised by 11.1% to ₹11.11 trillion", "category": "policy_impact"},
  {"title": "Foldable iPhone or iPad? Apple prepares to enter foldable device market by 2027", "category": "corporate_developments"},
  {"title": "Zuckerberg wins on Wall Street after Washington hit", "category": "corporate_developments"},
  {"title": "Keysight to Showcase Wireless Innovations at Mobile World Congress 2024", "category": "corporate_developments"},
  {"title": "Aster DM Healthcare to add 1,500 beds in 3 years, lines up Rs 850 crore investments", "category": "corporate_developments"},
  {"title": "15 Best Short Term Mutual Funds in India to Invest in April 2023", "category": "personal_finance"},
  {"title": "Respite For Paytm? Fintech Giant Set To Secure Govt Nod To Invest In Payments Gateway Arm", "category": "corporate_developments"},
  {"title": "How fast can I get an emergency loan?", "category": "personal_finance"},
  {"title": "HUDCO Standalone December 2023 Net Sales at Rs 2,012.66 crore, up 17.73% Y-o-Y", "category": "corporate_developments"},
  {"title": "White paper on UPA regime's mismanagement to be tabled soon: Finance Minister", "category": "policy_impact"},
  {"title": "Are Tax Brackets Adjusted for Inflation?", "category": "personal_finance"},
  {"title": "ITR filing: Income Tax department notifies ITR-2, ITR-3 forms for FY 2023-24. Details here", "category": "personal_finance"},
  {"title": "IIFL Securities, Angel One, ICICI Direct, other brokers hike margin on BNPL facility: Report", "category": "market_strategies"},
  {"title": "Rupa and Comp Standalone December 2023 Net Sales at Rs 312.74 crore, up 36.76% Y-o-Y", "category": "corporate_developments"},
  {"title": "PowerGrid trades 5% higher after Q3 net profit rises 10%", "category": "market_strategies"},
  {"title": "Large number of messages from bots, unrelated accounts: brokerage on #BoycottZerodha", "category": "market_strategies"},
  {"title": "ISPRL signs deal with Hindustan Petroleum to lease 300,000 MT space", "category": "corporate_developments"},
  {"title": "RBI Monetary Policy: FY24 inflation forecast unchanged at 5.4%; FY25 estimate at 4.5%", "category": "policy_impact"},
  {"title": "Civil aviation ministry to get Rs 2,300 cr budgetary allocation for 2024-25", "category": "policy_impact"},
  {"title": "WeWork founder Adam Neumann trying to buy back company", "category": "startup_ecosystem"},
  {"title": "Union Budget: FM announces plans to increase milk, dairy production", "category": "policy_impact"},
  {"title": "Budget 2024: Power PSUs NHPC, SJVN rally to record high after higher allocation", "category": "policy_impact"},
  {"title": "Tapestry lifts 2024 profit forecast on China demand rebound, strong margins", "category": "corporate_developments"},
  {"title": "Fed's Barkin says he's cautious about accuracy of recent economic data", "category": "economic_trends"},
  {"title": "Hybrid Financial Services reports consolidated net profit of Rs 0.32 crore in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Miner Freeport-McMoRan names insider Kathleen Quirk as CEO", "category": "corporate_developments"},
  {"title": "Viksit Bharat: Centre to provide Rs 75,000 crore interest-free loan to states", "category": "policy_impact"},
  {"title": "What is Sharpe Ratio in Mutual Fund and What is Its Significance?", "category": "personal_finance"},
  {"title": "Indraprastha Gas signs MoU to establish Compressed Biogas (CBG) plants", "category": "corporate_developments"},
  {"title": "'Paytm captain stands strong': Indian startups founders show support for Vijay Shekhar Sharma amid RBI's curbs", "category": "startup_ecosystem"},
  {"title": "Red Sea crisis disrupts Audi India's supply chain, firm eyes recovery soon", "category": "corporate_developments"},
  {"title": "Interim Budget: Infra focus, fiscal prudence are positives for street", "category": "policy_impact"},
  {"title": "Biggest consumer fintech trends to watch in 2024", "category": "market_strategies"},
  {"title": "Parliament Budget Session: India's coal production rises 10.3% to 99.73 MT in Jan 2024", "category": "economic_trends"},
  {"title": "Sterling & Wilson subsidiary mulls legal action for Australia contract termination", "category": "corporate_developments"},
  {"title": "E-commerce startup ShipBob seeks to hire banks for 2024 IPO -sources", "category": "startup_ecosystem"},
  {"title": "'India moving fast in defence production, 2,920 items indigenised so far'", "category": "policy_impact"},
  {"title": "FM Nirmala Sitharaman calls for more SBI-sized banks as India's economy grows", "category": "policy_impact"},
  {"title": "Budget: Govt panel to look into India's demographic challenges", "category": "policy_impact"},
  {"title": "Rajdarshan Ind Standalone December 2023 Net Sales at Rs 0.68 crore, up 69.53% Y-o-Y", "category": "corporate_developments"},
  {"title": "La Opala RG Standalone December 2023 Net Sales at Rs 107.14 crore, down 15.22% Y-o-Y", "category": "corporate_developments"},
  {"title": "Hudson Pacific Properties Reports Fourth Quarter 2023 Financial Results", "category": "corporate_developments"},
  {"title": "How Twitter's descent into chaos is paving the way for a new web", "category": "corporate_developments"},
  {"title": "Russia extends permit for LNG supplies to Germany's SEFE until 2040", "category": "corporate_developments"},
  {"title": "Competent Auto Standalone December 2023 Net Sales at Rs 669.30 crore, up 29.3% Y-o-Y", "category": "corporate_developments"},
  {"title": "D2C Brand Yu Raises Capital To Expand Food Products Portfolio", "category": "startup_ecosystem"},
  {"title": "Q3 results today: Maruti Suzuki, Sun Pharma, Ambuja Cements, Dabur, Bank of Baroda and more", "category": "corporate_developments"},
  {"title": "Inani Marbles Standalone December 2023 Net Sales at Rs 13.54 crore, up 23.96% Y-o-Y", "category": "corporate_developments"},
  {"title": "How to Get a Personal Loan without Income Proof?", "category": "personal_finance"},
  {"title": "Exclusive: Rubrik plans April listing amid U.S. probe into former employee -sources", "category": "market_strategies"},
  {"title": "Tata Chemicals Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "Fintech major Paytm denies any ED investigation into money laundering", "category": "corporate_developments"},
  {"title": "Stove Kraft Standalone December 2023 Net Sales at Rs 361.59 crore, up 11.42% Y-o-Y", "category": "corporate_developments"},
  {"title": "RBI MPC 2024: Governor Shaktikanta Das highlights initial nine points of the policy review", "category": "policy_impact"},
  {"title": "Buy TeamLease; target of Rs 3450: Motilal Oswal", "category": "market_strategies"},
  {"title": "PharmEasy-Owned Thyrocare To Acquire Think Health Diagnostics To Offer ECG Services At Home", "category": "corporate_developments"},
  {"title": "If There's 1 Must-Have Investment in Your Portfolio, It's This ETF", "category": "personal_finance"},
  {"title": "Veer Energy & Infrastructure standalone net profit declines 84.62% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Who gained and who lost in PM Modi’s last budget before polls", "category": "policy_impact"},
  {"title": "India's Paytm bank's slow drop-off in transactions may prompt deadline extension -sources", "category": "corporate_developments"},
  {"title": "Prime Focus Consolidated December 2023 Net Sales at Rs 840.85 crore, down 23.97% Y-o-Y", "category": "corporate_developments"},
  {"title": "9 in 10 Indian manufacturers expect higher production this quarter — domestic demand a key highlight: FICCI", "category": "economic_trends"},
  {"title": "RCF to roll out first prototype of Vande Metro coach by April: S Srinivas", "category": "corporate_developments"},
  {"title": "US farm exports hit three-year low in 2023 as China slows buying -Braun", "category": "economic_trends"},
  {"title": "Lancer Containe Standalone December 2023 Net Sales at Rs 104.83 crore, down 22.96% Y-o-Y", "category": "corporate_developments"},
  {"title": "Interim Budget 2024: Youth scaling new heights in sports - Sitharaman", "category": "policy_impact"},
  {"title": "Deductions on Salary Under Section 16 of the Income Tax Act, 1961", "category": "personal_finance"},
  {"title": "Swadeshi Polyte Standalone December 2023 Net Sales at Rs 54.55 crore, up 448.26% Y-o-Y", "category": "corporate_developments"},
  {"title": "Dividend news: Page Industries to give 3rd interim dividend of ₹100 per equity share", "category": "market_strategies"},
  {"title": "Know the incentives of those selling you a financial product – Varsity by Zerodha", "category": "personal_finance"},
  {"title": "Sensex falls amid weakness in banking stocks, analysts eye Nifty to reclaim 22,000 for renewed rally", "category": "market_strategies"},
  {"title": "Fed's Collins says her baseline expectation is 75 bps of rate cuts this year", "category": "economic_trends"},
  {"title": "Commercial Drone Alliance and Massachusetts Institute of Technology Lincoln Laboratory Announce Research Partnership", "category": "corporate_developments"},
  {"title": "Sinnar Bidi Standalone December 2023 Net Sales at Rs 1.21 crore, down 29.52% Y-o-Y", "category": "corporate_developments"},
  {"title": "India's January CPI inflation eases to 5.10%, core inflation drops to 3.6%", "category": "economic_trends"},
  {"title": "Berger Paints India Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "Will the Interim Budget 2024 give relief to debt mutual fund schemes?", "category": "policy_impact"},
  {"title": "ADM closes on flavor company purchase amid accounting probe", "category": "corporate_developments"},
  {"title": "SAIL shares fall 5% as Q3 profit suffers a 22% dent", "category": "market_strategies"},
  {"title": "Operating Leverage: What It Is, How It Works, How to Calculate", "category": "market_strategies"},
  {"title": "Nirmala Sitharaman invokes Vajpayee, Shastri's iconic slogan in Budget 2024 speech", "category": "policy_impact"},
  {"title": "Embraer, Mahindra sign deal to collaborate on C-390 transport aircraft", "category": "corporate_developments"},
  {"title": "Zee Entertain Standalone December 2023 Net Sales at Rs 1,914.20 crore, down 0.88% Y-o-Y", "category": "corporate_developments"},
  {"title": "Corporate Affairs Ministry To Review CCI’s Performance Amid Delay In Antitrust Probes", "category": "policy_impact"},
  {"title": "Skipper Q3 net profit jumps 115% to Rs 20.4 crore", "category": "market_strategies"},
  {"title": "India's United Breweries posts Q3 profit", "category": "market_strategies"},
  {"title": "Vision Sensing Acquisition Corp. Announces Payment of Fee to Extend Period to Consummate Initial Business Combination to March 3, 2024", "category": "market_strategies"},
  {"title": "ANMI approves proposal to extend trading hours for index futures: Report", "category": "market_strategies"},
  {"title": "India aims to raise up to $2.4 bn selling stakes in state-run firms", "category": "policy_impact"},
  {"title": "Budget 2024 highlights: Jal Marg Vikas Project allocation", "category": "policy_impact"},
  {"title": "Cyber Media (India) consolidated net profit declines 68.06% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "These Return Metrics Don't Make Kerjaya Prospek Group Berhad (KLSE:KERJAYA) Look Too Strong", "category": "market_strategies"},
  {"title": "Cracking the code: How to master UPI fraud management? MintGenie explains", "category": "personal_finance"},
  {"title": "Budget 2024: No profound impact on market; global cues to dictate trends, says Vinod Nair of Geojit Financial Services", "category": "market_strategies"},
  {"title": "Exicure, Inc. and Bluejay Therapeutics Inc. Enter into a Patent License Agreement to Develop Cavrotolimod for the Treatment of Hepatitis", "category": "corporate_developments"},
  {"title": "Builders seek interest rate cut in RBI's next meet to boost housing demand", "category": "policy_impact"},
  {"title": "Manappuram Fin Standalone December 2023 Net Sales at Rs 1,522.05 crore, up 22.46% Y-o-Y", "category": "corporate_developments"},
  {"title": "Prime Focus Standalone December 2023 Net Sales at Rs 7.12 crore, down 1.69% Y-o-Y", "category": "corporate_developments"},
  {"title": "How stock experts minted money from TV shows", "category": "market_strategies"},
  {"title": "BoI MF aims to raise Rs 500 cr from multi-asset allocation fund in NFO", "category": "personal_finance"},
  {"title": "NPCI appoints Ajay Kumar Choudhary as non-executive chairman of board", "category": "corporate_developments"},
  {"title": "Regent Ent Standalone December 2023 Net Sales at Rs 181.05 crore, down 11.44% Y-o-Y", "category": "corporate_developments"},
  {"title": "RBI clampdown on Paytm Payments Bank followed persistent non-compliance", "category": "corporate_developments"},
  {"title": "A quick guide to India's Interim Budget", "category": "policy_impact"},
  {"title": "20 Best Liquid Funds to Invest in India (April 2023)", "category": "personal_finance"},
  {"title": "Why JPMorgan Chase Is Banking on Opening 500 New U.S. Locations", "category": "corporate_developments"},
  {"title": "What is the Full Form of IFSC Code and How to Find it?", "category": "personal_finance"},
  {"title": "Luxury group Zegna to open new shoe and leather goods factory in Italy", "category": "corporate_developments"},
  {"title": "MSCI rejig on Feb 13: Jindal Stainless, PNB, BHEL, among others likely to enter global index", "category": "market_strategies"},
  {"title": "I-T dept collects over Rs 1,260 cr in TDS from online gaming, crypto trade", "category": "policy_impact"},
  {"title": "The US Is Investing $5B in Chip R&D—Here's What You Need to Know", "category": "policy_impact"},
  {"title": "What are Small-Cap Mutual Funds - Benefits, Taxation and Risks", "category": "personal_finance"},
  {"title": "India Budget: Manufacturing seeks business-friendly policies, thrust on innovation", "category": "policy_impact"},
  {"title": "Google To Spearhead AI-Led Development In Maharashtra", "category": "policy_impact"},
  {"title": "Why the e-bike boom is raising fire fears", "category": "market_strategies"},
  {"title": "Vibhor Steel Tubes garners Rs 22 cr from anchor investors", "category": "market_strategies"},
  {"title": "What are Indian Government Bonds - Types and Interest Rates", "category": "personal_finance"},
  {"title": "Short sellers take bets on nearly 8% of German bank PBB, filings show", "category": "market_strategies"},
  {"title": "CGI Inc. (TSE:GIB.A) Just Reported And Analysts Have Been Lifting Their Price Targets", "category": "market_strategies"},
  {"title": "Amazon to cut a few hundred jobs at healthcare units", "category": "corporate_developments"},
  {"title": "Unilever takes sales hit in Indonesia over anti-Israel boycotts", "category": "corporate_developments"},
  {"title": "India's retail inflation eases to 3-month low of 5.1% in January 2024", "category": "economic_trends"},
  {"title": "Shree Kr Paper Standalone December 2023 Net Sales at Rs 25.97 crore, down 36.49% Y-o-Y", "category": "corporate_developments"},
  {"title": "Stock market today: Why Bank stocks are rising ahead of US CPI data release — explained with two reasons", "category": "market_strategies"},
  {"title": "FPIs on bullish stance; infuse over Rs 15,000 cr in debt market in Feb | News on Markets - Business Standard", "category": "market_strategies"},
  {"title": "Reliance Industries becomes India's first ₹20 lakh crore company; stock at record high", "category": "market_strategies"},
  {"title": "Gokaldas Exports completes acquisition of Coral Investments", "category": "corporate_developments"},
  {"title": "Sharif Party to Hold Talks With Rival on Pakistan Coalition", "category": "economic_trends"},
  {"title": "Interim Budget FY24-25 — why it conveys a clear message for the upcoming elections", "category": "policy_impact"},
  {"title": "Wires and Fabri Standalone December 2023 Net Sales at Rs 29.02 crore, up 2.91% Y-o-Y", "category": "corporate_developments"},
  {"title": "5 takeaways from the Interim Budget 2024 for investors", "category": "market_strategies"},
  {"title": "Ageas Federal Life Insurance launches Multicap Fund in ULIP portfolio", "category": "personal_finance"},
  {"title": "Vibhor Steel Tubes IPO fully subscribed: GMP, subscription status, review, other details in 10 points. Apply or not?", "category": "market_strategies"},
  {"title": "Joe Biden Wants to Make Big Changes to Social Security: 10 Things You Need to Know", "category": "economic_trends"},
  {"title": "PayPal Stock Slides as Company Transformation Weighs on Guidance—Key Price Levels to Watch", "category": "market_strategies"},
  {"title": "Congo's Gecamines and Entreprise Generale du Cobalt sign mining deal", "category": "corporate_developments"},
  {"title": "What Is Prime Cost? Definition, Formula, Calculation, and Purpose", "category": "market_strategies"},
  {"title": "Apollo Tyres Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "India aims to lower fiscal deficit to 5.8% this year, expects 10.5% nominal GDP growth in dollar terms", "category": "policy_impact"},
  {"title": "Paushak standalone net profit declines 9.25% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Polychem Standalone December 2023 Net Sales at Rs 9.48 crore, up 11.81% Y-o-Y", "category": "corporate_developments"},
  {"title": "UPI, RuPay services to be launched in Sri Lanka, Mauritius on Monday", "category": "corporate_developments"},
  {"title": "Do you evaluate before availing a loan?", "category": "personal_finance"},
  {"title": "Air India launches Namaste World Sale. Price, date, other details", "category": "market_strategies"},
  {"title": "Can go to NCLT to enforce Sony merger deal after SIAC rejects petition: Zee", "category": "corporate_developments"},
  {"title": "VRL Logistics standalone net profit declines 72.26% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Opinion: Sitharaman's growth push finds a dependable ally in RBI", "category": "policy_impact"},
  {"title": "Goldman Sachs Fund Unit Finds Value After Green Market Rout", "category": "market_strategies"},
  {"title": "Italy readies targeted tax breaks for farmers, minister says", "category": "policy_impact"},
  {"title": "Creative New Standalone December 2023 Net Sales at Rs 493.06 crore, up 20.25% Y-o-Y", "category": "corporate_developments"},
  {"title": "What are the Advantages and Disadvantages of Credit Card?", "category": "personal_finance"},
  {"title": "Anarock’s expansion plan: Setting up office brokerage platform with ₹200 crore investment", "category": "corporate_developments"},
  {"title": "What is Treasury Bill (T-Bill)? - Meaning, Types, Features and Benefits", "category": "personal_finance"},
  {"title": "Interim Budget: Ayushman Bharat cover to be extended to all ASHA workers", "category": "policy_impact"},
  {"title": "Saudi Arabia considers $10 bln Aramco share sale, Bloomberg reports", "category": "market_strategies"},
  {"title": "Hyundai picks JP Morgan, Citi and HSBC as advisors for mega IPO of India arm", "category": "market_strategies"},
  {"title": "RBI action against Paytm will hit ordinary Indians most: Cong slams govt", "category": "policy_impact"},
  {"title": "Budget 2024: Tax exemption deadline extended by one year for these categories", "category": "policy_impact"},
  {"title": "Star Health Standalone December 2023 Net Sales at Rs 3,578.35 crore, up 16.46% Y-o-Y", "category": "corporate_developments"},
  {"title": "RBI on Paytm Payments Bank: Not a case of regulatory deficiency, but issue of compliance, says Governor Das", "category": "policy_impact"},
  {"title": "Indonesia 2023 GDP growth slows to 5.1% on falling commodity prices", "category": "economic_trends"},
  {"title": "United Drilling Standalone December 2023 Net Sales at Rs 34.04 crore, up 28.42% Y-o-Y", "category": "corporate_developments"},
  {"title": "GST fake invoices: Govt to do away with edit option to revise output liability, says Revenue Secretary", "category": "policy_impact"},
  {"title": "Neil Diamond’s \"I Am… I Said\" Featured in Volkswagen’s \"An American Love Story\" Extended Cut in Advance of Super Bowl LVIII Advertisement", "category": "corporate_developments"},
  {"title": "VeriSign Inc Reports Solid Financial Growth Amidst Global Economic Challenges", "category": "corporate_developments"},
  {"title": "Housing Expense Ratio: What it is, How it Works", "category": "personal_finance"},
  {"title": "Paytm Bank fiasco: Mutual funds that have highest and lowest exposure to Vijay Shekhar Sharma's Paytm", "category": "market_strategies"},
  {"title": "Olympic Fin Ser Standalone December 2023 Net Sales at Rs 0.02 crore, down 6.59% Y-o-Y", "category": "corporate_developments"},
  {"title": "Mahadev Online Book Promoters Under DGGI Scanner For GST Dues Worth INR 30,000 Cr", "category": "policy_impact"},
  {"title": "SoftBank-backed Meesho unveils Valmo to enable low-cost deliveries", "category": "startup_ecosystem"},
  {"title": "No fixed disinvestment target in FY25 Budget, says DIPAM Secretary", "category": "policy_impact"},
  {"title": "Paytm Money rebuffs media report on CDSL's scrutiny of KYC process, says it is 'routine audit’", "category": "corporate_developments"},
  {"title": "Cabinet approves six multi-tracking projects worth Rs 12,000 crore", "category": "policy_impact"},
  {"title": "Sovereign Gold Bonds 2016 Series I to give 101% profit at maturity: Check final redemption date", "category": "personal_finance"},
  {"title": "State Bank of India eyes 12-13% growth in international loan book", "category": "corporate_developments"},
  {"title": "Tax on Dividend Income: Sources, Tax Rate and TDS on dividend income", "category": "personal_finance"},
  {"title": "Digjam Standalone December 2023 Net Sales at Rs 5.78 crore, down 35.27% Y-o-Y", "category": "corporate_developments"},
  {"title": "Week Ahead: Inflation data, Q3 results, global cues among key market triggers this week", "category": "market_strategies"},
  {"title": "Virgin Galactic reports flight-related issue to FAA", "category": "corporate_developments"},
  {"title": "Eicher Motors Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "Thai PM: will keep urging c.bank to cut rates", "category": "economic_trends"},
  {"title": "China’s small-cap crash shows what happens without market rescue", "category": "market_strategies"},
  {"title": "From ABBA to SRK: 3 hidden money lessons from movies and music to bloom this Valentine's Day 2024", "category": "personal_finance"},
  {"title": "PM-KISAN e-KYC deadline today! Here's how to secure your ₹2000 instalment", "category": "policy_impact"},
  {"title": "BJP vs Congress: Centre refutes Karnataka's claims as face-off over tax devolution heats up", "category": "policy_impact"},
  {"title": "Why Steris (STE) is a Top Growth Stock for the Long-Term", "category": "market_strategies"},
  {"title": "Indusind Bank Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "Centre's white paper lists how Modi govt 'rescued economy from state of crisis'", "category": "policy_impact"},
  {"title": "What happened to Paytm Payments Bank? RBI's clampdown explained", "category": "corporate_developments"},
  {"title": "Budget and its numbers: A breakdown of key estimates for FY25", "category": "policy_impact"},
  {"title": "Budget 2024 allocates Rs 57 crore for drone and drone components under PLI scheme", "category": "policy_impact"},
  {"title": "Viacom18 expands horizon, tunes into live events", "category": "corporate_developments"},
  {"title": "BS-IMGC India conclave: Redefining role of financiers in housing ecosystem", "category": "policy_impact"},
  {"title": "Software firm Everbridge to be taken private by Thoma Bravo in $1.5 bln deal", "category": "market_strategies"},
  {"title": "Liquidity Adjustment Facility (LAF): Meaning, Importance and Roles", "category": "economic_trends"},
  {"title": "Bunge posts profit beat, warns weaker margins to dent 2024 results", "category": "corporate_developments"},
  {"title": "ISEE Commercially Deploys World’s First Fully Autonomous Truck Yard", "category": "corporate_developments"},
  {"title": "MM Forgings Standalone December 2023 Net Sales at Rs 387.75 crore, up 7.39% Y-o-Y", "category": "corporate_developments"},
  {"title": "EaseMyTrip Eyes Foray Into Hospitality Space With 5-Star Hotel In Ayodhya", "category": "corporate_developments"},
  {"title": "Natural Capsules standalone net profit declines 67.62% in the December 2023 quarter", "category": "corporate_developments"},
  {"title": "Why the $5.2 billion sale of Russia's Yandex is significant", "category": "market_strategies"},
  {"title": "Market Action: 4 IPOs to open for subscription, 9 companies scheduled for debut on bourses next week", "category": "market_strategies"},
  {"title": "US recall says Tesla brake warnings were too small", "category": "corporate_developments"},
  {"title": "With neutral policy tone & liquidity management, rate cut unlikely any time soon", "category": "economic_trends"},
  {"title": "Marketbuzz Podcast with Kanishka Sarkar: Here are 10 key talking points", "category": "market_strategies"},
  {"title": "Interim Budget makes no attempt to woo voters, says Harsh Gupta", "category": "policy_impact"},
  {"title": "Scandal-hit business group will survive, says new boss", "category": "corporate_developments"},
  {"title": "GST collections to drive FY25 gross tax revenue to Rs 38.31 trillion", "category": "policy_impact"},
  {"title": "Ryanair CEO: Boeing quality is a concern, further delays possible", "category": "corporate_developments"},
  {"title": "Fed Chair Powell faces tough communications task on rate cuts ahead", "category": "economic_trends"},
  {"title": "AbbVie Cuts Its Guidance on ImmunoGen Deal", "category": "corporate_developments"},
  {"title": "Jana Small Finance Bank IPO opens next week. Price band fixed at ₹393 to ₹414 per share", "category": "market_strategies"},
  {"title": "FM Sitharaman allocates Rs 13,042 crore to Department of Space in interim Budget, marking 18% increase", "category": "policy_impact"},
  {"title": "IMF's Kristalina Georgieva says she's confident on global economic outlook despite uncertainties", "category": "economic_trends"},
  {"title": "66 Nifty 500 stocks rallied between 20% and 104% in CY24 so far; NBCC, SJVN, UCO Bank among top performers", "category": "market_strategies"},
  {"title": "How to Check Your TDS Status by Using Your PAN Card?", "category": "personal_finance"},
  {"title": "Cochin Minerals Standalone December 2023 Net Sales at Rs 82.16 crore, down 30.98% Y-o-Y", "category": "corporate_developments"},
  {"title": "Ralph Lauren rides strong holiday demand, China rebound to post upbeat results", "category": "corporate_developments"},
  {"title": "Could a standalone non-profit be the answer to FAA oversight of Boeing?", "category": "policy_impact"},
  {"title": "Tesla opens first South America store in Chilean capital", "category": "market_strategies"},
  {"title": "Ukraine's farm unions ask government to protect free EU market access", "category": "policy_impact"},
  {"title": "Sovereign Gold Bond Scheme 2023-24 Series IV opens — a look at historical returns", "category": "personal_finance"},
  {"title": "Bayer CEO says company stands behind glyphosate", "category": "corporate_developments"},
  {"title": "Paytm wallet business at risk as licence transfer uncertain: Report", "category": "corporate_developments"},
  {"title": "Budget 2024: Govt allocates ₹1,200 crore for North-east gas subsidy scheme", "category": "policy_impact"},
  {"title": "Bank of Canada says more time needed to bring down inflation", "category": "economic_trends"},
  {"title": "Ucal opens an after-market division", "category": "corporate_developments"},
  {"title": "India's Ola Electric considers bid for lithium mining rights, sources say", "category": "corporate_developments"},
  {"title": "Budget 2024 Key Highlights LIVE Updates: No changes in direct, indirect tax rates; FY25 fiscal deficit target at 5.1%", "category": "policy_impact"},
  {"title": "Zambian businessman loses London lawsuit against Bob Diamond's Atlas Mara", "category": "economic_trends"},
  {"title": "Karnataka HC tells state to recover stuck homebuyers' money within 4 months", "category": "policy_impact"},
  {"title": "Shinjini Kumar resigned from PPBL board, attended 2 meets as special guest", "category": "corporate_developments"},
  {"title": "Disney answers activists with gaming investment, ESPN streaming plans", "category": "corporate_developments"},
  {"title": "Budget 2024 showcases positive macro picture: Fiscal recovery, capex-led growth, and surging corporate profits", "category": "policy_impact"},
  {"title": "Different Types of Debit Cards in India", "category": "personal_finance"},
  {"title": "Tax Lien Foreclosure: Definition, How It Works, Vs. Tax Deed Sale", "category": "personal_finance"},
  {"title": "RBI Policy: India's real GDP growth for FY25 projected at 7%", "category": "policy_impact"},
  {"title": "Vascon Engineer Consolidated December 2023 Net Sales at Rs 280.25 crore, up 10.33% Y-o-Y", "category": "corporate_developments"},
  {"title": "Is Cyprus Considered a Tax Haven?", "category": "economic_trends"},
  {"title": "Petrobras in talks for projects with Gulf, Indian, Chinese firms, CEO says", "category": "corporate_developments"},
  {"title": "Cummins urges continued infrastructure spending and GST subsidies in Budget 2024", "category": "policy_impact"},
  {"title": "Shri Jagdamba Standalone December 2023 Net Sales at Rs 70.61 crore, up 8.7% Y-o-Y", "category": "corporate_developments"},
  {"title": "RBI keeps repo rate unchanged. What does that mean for your home loan EMI", "category": "policy_impact"},
  {"title": "Beware of Pig Butchering Scams – Varsity by Zerodha", "category": "personal_finance"},
  {"title": "Jeff Bezos Sells Nearly 12 Million Amazon Shares Worth at Least $2 Billion", "category": "market_strategies"},
  {"title": "Short Run and Long Run Cost", "category": "market_strategies"},
  {"title": "Mallcom (India) Standalone December 2023 Net Sales at Rs 92.01 crore, down 3.4% Y-o-Y", "category": "corporate_developments"},
  {"title": "LIC Housing Finance expects to report 15% improvement in FY25 loan book", "category": "market_strategies"},
  {"title": "Brightcom Group Share Price Live blog for 13 Feb 2024", "category": "market_strategies"},
  {"title": "ONGC in talks to restart Libya operations, expand in Venezuela: Report", "category": "corporate_developments"},
  {"title": "Sovereign Gold Bond: Various investment platforms offer options to invest in SGB; how it is compared to physical gold", "category": "personal_finance"},
  {"title": "Heartland Express Inc (HTLD) Reports Record Annual Operating Revenue Amidst Industry Challenges", "category": "corporate_developments"},
  {"title": "DMR Hydroengineering & Infrastructures invests subscribes to rights issue of DM Consulting", "category": "corporate_developments"},
  {"title": "Kazakhstan's Air Astana raises $120 mln in dual London, local IPO", "category": "market_strategies"}'''

2024-02-15 12:40:52,966| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [62]:
import json

In [93]:
res = responses_from_gpt.replace('\n','')

In [94]:
k = res.split('},')

In [95]:
k1 = []
for x in k:
    try:
        k1.append(json.loads(x + '}'))
    except:
        pass

In [97]:
dfk = pd.DataFrame(k1)

2024-02-15 12:42:03,463| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [98]:
dfk.groupby('category')['title'].count()

category
corporate_developments    105
economic_trends            18
market_strategies          58
personal_finance           29
policy_impact              62
startup_ecosystem           6
Name: title, dtype: int64

In [ ]:
dfk[dfk.category == 'economic_trends']

In [88]:
len(k1)

213

2024-02-15 12:39:22,316| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [81]:
k = [json.loads(x + '}') for x in k ]

JSONDecodeError: Expecting ',' delimiter: line 1 column 37 (char 36)

2024-02-15 12:38:01,714| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-15 12:38:11,777| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [79]:
json.loads(k[0] + '}')

{'title': 'A.K.Capital Ser Standalone December 2023 Net Sales at Rs 31.44 crore, up 7.24% Y-o-Y',
 'category': 'corporate_developments'}

2024-02-15 12:37:31,480| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-15 12:37:41,567| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [64]:
json.loads(responses_from_gpt)

JSONDecodeError: Expecting ',' delimiter: line 3 column 37 (char 313)

2024-02-15 12:35:10,524| ERROR   | Could not establish connection from local ('127.0.0.1', 3712) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [51]:
df[df.source_name == 'zerodha-varsity']

,article_id,published_at,financial_news,relevant_for_india,india_flag,source_name,title
49,652ebbe41e5cc42b1b139c1f,2023-02-16 11:36:31+00:00,True,True,None,zerodha-varsity,"14. Supplementary note – Rights, OFS, FPO ..."
288,652ebbda1e5cc42b1b139b56,2023-04-20 06:36:51+00:00,True,True,None,zerodha-varsity,15. Supplementary note – The 20 market depth...
391,65b0b5956391ac19baa878e8,2024-01-24 06:36:21+00:00,True,True,indian,zerodha-varsity,Things to look for in the (Interim) Budget – V...
595,652ebbdc1e5cc42b1b139b75,2023-09-13 12:16:41+00:00,True,True,None,zerodha-varsity,10. Clearing and Settlement Process : Intr...
757,6538f5501e5cc42b1b1444a9,2023-10-25 08:12:00+00:00,True,True,None,zerodha-varsity,What is Venture Capital? – Varsity by Zerodha
834,652ebbe41e5cc42b1b139c29,2023-07-24 07:00:23+00:00,True,True,None,zerodha-varsity,4. The IPO Markets (Part 1) : Introduction...
1038,652ebbe11e5cc42b1b139bd2,2023-10-13 09:53:01+00:00,True,True,None,zerodha-varsity,10. Steel (Part 1) : Sector Analysis
1051,65a1fcd1e362d630b6473533,2024-01-12 13:07:32+00:00,True,True,indian,zerodha-varsity,Modes of raising funds : Social Stock Exchange...
1178,652ebbe01e5cc42b1b139bb3,2023-08-10 10:52:53+00:00,True,True,None,zerodha-varsity,29. Exchange-traded funds (ETF) : Personal...
1335,65570f8e4b13023f934aee90,2023-11-17 05:36:44+00:00,True,True,None,zerodha-varsity,How to forecast the economy? Economic Indicato...


In [28]:
df.groupby('india_flag')['article_id'].count()

india_flag
Indian            109
International       2
indian           2905
international    1402
Name: article_id, dtype: int64

In [22]:
articles['652ebb751e5cc42b1b1398c3']

{'_id': ObjectId('652ebb751e5cc42b1b1398c3'),
 'url': 'https://zerodha.com/varsity/stock-market-scams-and-how-to-avoid-them-pump-and-dump/',
 'source_id': ObjectId('65291b1e9a2fbc229e5f29c9'),
 '_class': 'com.wintWealth.insight.common.models.entities.Article',
 'image_url': 'https://zerodha.com/varsity/wp-content/uploads/2023/10/Frame-623-300x171.png',
 'is_premium_article': False,
 'last_updated_time': '2023-10-04T06:30:32+0000',
 'published_time': '2023-10-04T06:30:32+0000',
 'short_description': 'Several stocks have given 1000% returns within a few months. Who could have predicted that kind of rally for any stock? Hardly anyone. Most rallies are a surprise. However, many people do predict massive gains for various stocks. Several channels and groups on YouTube, WhatsApp, and Instagram make tall predictions.\xa0 This ₹20 stock will go […]',
 'tags': ['Equity Investing'],
 'title': 'Stock market scams and how to avoid them - Pump and Dump – Varsity by Zerodha',
 'authors': ['Vineet Ra

In [16]:
df.groupby('source_name')['article_id'].count()

source_name
bbc                    52
business-standard     960
cnbc-tv-18            413
economic-times         30
finshots               10
groww                  34
inc42                 134
investopedia          158
live-mint             470
money-control        1174
navi                  180
reuters               789
wint-wealth             8
yahoo-finance         222
zerodha-varsity        32
Name: article_id, dtype: int64